In [2]:
import pandas as pd
from transformers import pipeline


StatementMeta(, e74fc30a-ad05-46e7-acbe-140b67512421, 4, Finished, Available, Finished)

In [3]:
df = spark.read.format("csv") \
    .option("header","true") \
    .load("abfss://Testing@onelake.dfs.fabric.microsoft.com/TestLake.Lakehouse/Files/nps_result.csv")
# df now is a Spark DataFrame containing CSV data from "abfss://Testing@onelake.dfs.fabric.microsoft.com/TestLake.Lakehouse/Files/nps_result.csv".
display(df)

StatementMeta(, e74fc30a-ad05-46e7-acbe-140b67512421, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b769c81a-ee9f-49a6-8816-ae1d2ea3d676)

In [4]:
# Convertir a pandas
df_pd = df.toPandas()
# Verificá si el campo "score" está como string o número
df_pd["score"] = pd.to_numeric(df_pd["score"], errors="coerce")
# Ya podés usar df_pd con Hugging Face u otras herramientas Python
df_pd.head()

StatementMeta(, e74fc30a-ad05-46e7-acbe-140b67512421, 6, Finished, Available, Finished)

,survey_start_date,survey_end_date,chain_id,business_id,score,category,date,country_id,order_id,comment,WHS_name
0,2022-06-28 22:00:00.000,2022-12-28 22:00:00.000,87,5855,10,Promoter,2022-02-12 14:19:30.000,10,18735,??????,Skappie's Place TEST LOYALTY
1,2021-07-01 05:00:00.000,2024-06-30 06:00:00.000,25,2034,8,Passive,2021-12-30 02:46:01.000,12,6055,es seguro,Bodegas Alianza
2,2021-07-28 16:00:00.000,2030-01-16 16:00:00.000,77,5929,7,Passive,2021-10-13 06:32:40.000,13,10797,No reason,Platinum/ Royalton
3,2021-07-27 16:00:00.000,2026-01-07 16:00:00.000,71,6840,9,Promoter,2021-12-24 07:49:21.000,13,15756,ok,Sunbear
4,2022-06-28 22:00:00.000,2022-12-28 22:00:00.000,87,5866,10,Promoter,2022-02-17 18:00:44.000,10,19028,excellent,Skappie's Place TEST LOYALTY


In [5]:

# 2. Agrupar comentarios por 'score' concatenando los comentarios, ignorando valores irrelevantes (algunos de ejemplo..)
agrupado = df_pd.groupby("score")["comment"].apply(
    lambda x: " ".join(str(c) for c in x if str(c).strip().lower() not in ["na", "??", ".", "...", ""])
).reset_index()

# 3. Crear pipeline de resumen (usa modelo pre-entrenado)
# Importa y crea un pipeline de resumen usando el modelo facebook/bart-large-cnn, que es un modelo de lenguaje entrenado para tareas de resumen de texto.
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# 4. Función para resumir texto, manejando errores y límite de tokens
def resumir_comentarios(texto):
    # si no hay texto, rta:
    if len(texto.strip()) == 0:
        return "No hay comentarios relevantes."
    try:
        max_chunk = 1000
        texto_proc = texto[:max_chunk]
        resumen = summarizer(texto_proc, max_length=150, min_length=40, do_sample=False)
        return resumen[0]['summary_text']
    except Exception as e:
        return f"Error al resumir: {e}"

# 5. Aplicar resumen para cada grupo de score
# aplica función por score
agrupado["resumen"] = agrupado["comment"].apply(resumir_comentarios)




StatementMeta(, e74fc30a-ad05-46e7-acbe-140b67512421, 7, Finished, Available, Finished)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

StatementMeta(, e74fc30a-ad05-46e7-acbe-140b67512421, 10, Finished, Available, Finished)

In [6]:
# 6. resultados
pd.set_option('display.max_colwidth', None)
agrupado[["score", "resumen"]]

StatementMeta(, e74fc30a-ad05-46e7-acbe-140b67512421, 8, Finished, Available, Finished)

,score,resumen
0,1,No llega el pedido bn falta de descripcion de productos. no me llagan los pedios es muy molesta esta encuesta .... no es practica mala. No llego lo that pedí hace uno los pedidos and no llegan.
1,2,"Falta bastante produco no avisan los agotados, no cumplen with las politicas de entrega. Is it possible that when we collect the order we find the order already processed just waiting for payment and collection?"
2,3,No siempre se cumple with las fechas y horarios sugeridos. Always deliver items not ordered slow and doesnt stock all products. incorrect product descriptions t N/A.
3,4,The app is very slow and nice. Sometimes very hard to order once. Need to remove the date of order delivery thing uts unnecessary for now. Pregunta mucho calificación too slowly gf ooo ll hhh.
4,5,Falta mucho producto solo por la stencion del preventa. No hay promociones atractivas los precios de lista estan elevados. Todavía les falta n productos and poner cantidades.
5,6,Aplicación es bueno pero no deja pedir muchas ofertas buena. Falta mucho producto pero el servicio that da el preventa es muy bueno. He tenido problemas ya que muchas de las veces que intento buscar algo se congela la pagi.
6,7,"Por la. Atención del. Ejecutivo gg muy bueno es bueno pero falta mas aclarida. No me han llegado los pedidos a tiempo. si Excelente , Buenas promociones j bien. No reason s g con la ayuda del ejecUTivo es facil entender la app."
7,8,"La aplicacion es facil de usar ya que el ejecutivo enseña muy bien su funcionamiento. No tienen todos los productos incluidos en su sistema porque les falta portafolio, y se enfocan demasiado en Pernod. No se demoran en llegar los pedidos buena"
8,9,Good app megusta la forma de información que nos ofrecen. Buenas ofertas por el excelente manejo y promoción. Facil uso entrega contra pago h muy práctico para hacer los pedidos es seguro.
9,10,Aplicacion promociones y envio gratis. ?????? excellent si Facilidad ???? Happy with the service promo good buenas ofertas porque es bueno es facil de administrar efectivo y rapido.


In [7]:
# comparo un score con lo que se ve full 
df_pd[df_pd["score"] == 3]["comment"]

StatementMeta(, e74fc30a-ad05-46e7-acbe-140b67512421, 9, Finished, Available, Finished)

256                                                                                    si
303                                                                   le faltan productos
351           aveces se cumple con la entrega como aveces la cumplen a toda satisfacción 
609                             No siempre se cumple con las fechas y horarios sugeridos 
1264                                                    la aplicación tiene muchos fallos
1307                                                        no me gusta que me pregunten 
1312                la mayoria deproductos no se encuentran disponibles en la aplicación 
1383                                                                         troublesome 
1579                                                                            VERY nice
1702                                                                                 good
1710                                                                                none 
1739      